In [146]:
import os
# os.chdir('C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol')
os.chdir('/home/centos/Alcohol-on-Twitter')

import pickle
import csv
import json
import sklearn
print(sklearn.__version__)
import pandas as pd
import numpy as np
import sys
from evaluation.metric import computeAccuracy, computeF1Score, computeAUC
# from sklearn.metrics import roc_auc_score

# %load_ext autoreload
# %autoreload
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from scipy.stats import uniform
# %reload_ext pipelines.alcohol.AlcoholPipeline
from pipelines.alcohol import AlcoholPipeline
from pipelines.ecig import ECigPipeline

0.21.3


In [147]:
# # test ----------
# # df_test = pd.read_csv("./data/alcohol_training_instances.csv")
# df_test = pd.read_csv("/mnt/volume/result/alcohol_training_instances.csv")
# df_test[:3]

In [148]:
# read the data
# filePath = "C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol/data/labeled_figure_eight_csv_1000.csv"
filePath = "/mnt/volume/labeled_data/labeled_figure_eight_csv_1000.csv"
df = pd.read_csv(filePath)
df.is_this_posting_relevant_to_ecigarette[:3]

# Deal with ecig_labels.
ecig_labels = []
# extracting pre-labeled classification
for i in range(0,len(df.is_this_posting_relevant_to_ecigarette)):
#     print(df.is_this_posting_relevant_to_ecigarette[i])
    if df.is_this_posting_relevant_to_ecigarette[i] == 'ECIG':
        ecig_labels.append(1)
    elif df.is_this_posting_relevant_to_ecigarette[i] == 'NOT-ECIG':
        ecig_labels.append(0)
    else:
        print('Should not go to here')
# ecig_labels


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.realdonaldtrump_foxandfriends_smoke_blowing_up_your_orange_ass, ecig_labels, test_size=0.2, random_state=26)
print('length of X_train: ', len(X_train))
print('length of X_test: ', len(X_test))
print('length of y_train: ', len(y_train))
print('length of y_test: ', len(y_test))
# X_train
# y_train


# # test ----------
# df_test = pd.read_csv("./data/alcohol_training_instances.csv")
# df_test
# df_test.labels[1]

# alc_labels = []
# # extracting pre-labeled classification
# for i in range(0,len(df_test.labels)):
#     alc_labels.append(df_test.labels[i][12])
# alc_labels = list(map(int, alc_labels))

# from sklearn.cross_validation import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(df_test, alc_labels, test_size=0.1, random_state=26)
# X_train

length of X_train:  801
length of X_test:  201
length of y_train:  801
length of y_test:  201


In [149]:
sum(ecig_labels)

571

In [150]:
# LR
clf_ecig_LR = ECigPipeline().pipeline(LogisticRegression())
clf_ecig_LR.steps

[('tfidf',
  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                  dtype=<class 'numpy.float64'>, encoding='utf-8',
                  input='content', lowercase=True, max_df=1.0, max_features=None,
                  min_df=1, ngram_range=(1, 3), norm='l2', preprocessor=None,
                  smooth_idf=True, stop_words=None, strip_accents=None,
                  sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                  tokenizer=None, use_idf=True, vocabulary=None)),
 ('clf',
  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                     intercept_scaling=1, l1_ratio=None, max_iter=100,
                     multi_class='warn', n_jobs=None, penalty='l2',
                     random_state=None, solver='warn', tol=0.0001, verbose=0,
                     warm_start=False))]

In [151]:
# Fit the data into model to see the performance.

In [152]:
clf_ecig_LR.fit(X_train, y_train)

/home/centos/anaconda3/envs/yijun/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 3), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling

In [153]:
# predict in testing set.
y_ecig_LR = clf_ecig_LR.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_ecig_LR, y_test)
f1_score_LR = computeF1Score(y_test, y_ecig_LR)
print('F1 Score: ', f1_score_LR)
auc_score_LR = computeAUC(clf_ecig_LR, X_test, y_test, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('alcohol LR AUC: ', auc_score_LR)

# confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_ecig_LR)

length of total comps:  201
62 different item in whole dataset
accuracy: 69.154229%
F1 Score:  0.75
alcohol LR AUC:  0.8499701729966197


array([[46, 48],
       [14, 93]])

We can see the results is fine. Now we will use this classifier on our San Francisco data to see how many of them are e-cig related.

In [154]:
SF_filePath = '/mnt/volume/result/juliana_allSF_2018_01.json'
df_SF_01 = pd.read_json(SF_filePath, lines=True, encoding='utf-8') 


In [155]:
# df_SF_01[:3]

In [156]:
df_SF_01.text[:3]

0    Still totally love my boyfriend even if he can...
1      @daniellenatoli1 @ me in a pic &amp; then maybe
2    @imillhiser @IPM_GS @benwikler @RVAwonk @AdamP...
Name: text, dtype: object

In [157]:
type(df_SF_01.text)

pandas.core.series.Series

In [158]:
print('length of San Francisco data: ', len(df_SF_01))
y_SF_LR = clf_ecig_LR.predict(df_SF_01.text)
print('How many of them the classifier said is positive (e-cig relevant)?' % (sum(y_SF_LR)))

length of San Francisco data:  278884
How many of them the classifier said is positive (e-cig relevant)?


In [22]:
y_SF_LR[:20]

array([1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0])

In [23]:
for text in df_SF_01.text[:20]:
    print(text)
    print('------------')

Still totally love my boyfriend even if he can't ride a bike. We'll take the bus... https://t.co/c1S1OTqPfA
------------
@daniellenatoli1 @ me in a pic &amp; then maybe
------------
@imillhiser @IPM_GS @benwikler @RVAwonk @AdamParkhomenko @SarahLerner @brianbeutler @cristianafarias @jdawsey1… https://t.co/Bf2nsE2POu
------------
@lilnatttt make your mind up, i need you to make your mind up
------------
@NatGeo .@NatGeo's book "Those Inventive Americans" emphasizes how original some human beings are and how their cre… https://t.co/Ojyp84kFGU
------------
te amo te amo te amo https://t.co/anZ0k4S8xy
------------
@her_na_lnr Меня уже ничего не удивляет. Лучше бы легализировали LSD.
------------
Most copped on Restocks last week?
------------
@matthewherper @daviesbj @jgreid @adamfeuerstein @Dr__Guess @YoniFreedhoff And that says a lot
------------
Jeremy Lamb shooting 2/3 from 3pt range. 8 points showing some good spot up shooting.
------------
TEST_LAT/LON: 7f521b31-f8d8-4c2e-85d2-ddb826

In [24]:
clf_ecig_LR.steps

[('tfidf',
  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                  dtype=<class 'numpy.float64'>, encoding='utf-8',
                  input='content', lowercase=True, max_df=1.0, max_features=None,
                  min_df=1, ngram_range=(1, 3), norm='l2', preprocessor=None,
                  smooth_idf=True, stop_words=None, strip_accents=None,
                  sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                  tokenizer=None, use_idf=True, vocabulary=None)),
 ('clf',
  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                     intercept_scaling=1, l1_ratio=None, max_iter=100,
                     multi_class='warn', n_jobs=None, penalty='l2',
                     random_state=None, solver='warn', tol=0.0001, verbose=0,
                     warm_start=False))]

In [26]:
clf_ecig_LR.steps
LR_ecig = clf_ecig_LR.steps[1][1]
coefficients = LR_ecig.coef_.tolist()[0]
print('length of coefficients: ', len(coefficients))

In [28]:
tfidf_ecig = clf_ecig_LR.get_params()['tfidf']
features = tfidf_ecig.get_feature_names()
print('length of features: ', len(features))
type(features)

length of features:  19904


list

In [29]:
featuresAndCoef = {'features':features,'coefficients':coefficients}
df_featuresAndCoef = pd.DataFrame(featuresAndCoef)
df_featuresAndCoef.sort_values(by=['coefficients'], ascending=False)

,features,coefficients
8951,juul,3.204274
17589,vape,2.814391
18016,vaping,1.738162
11014,my,1.016856
11099,my juul,0.839300
12920,pods,0.780632
16015,the juul,0.728589
9073,juul pods,0.634277
10488,me,0.615104
2878,can,0.579114


Even the coefficients seem good, we still got 201606 out of 278884 the classifier said was positive (e-cig relevant), which is not good. How about we include some negative sample?

In [130]:
def checkECigaretteInLine(line):
    lineInJson = json.loads(line)
    textInLine = lineInJson["text"]
    lowercaseLine = textInLine.lower()

    # Category keyword
    synonym_ecig = ['electronic-cigarette', 'electronic cigarette', 'electronic cig', ' e-cig', ' ecig', ' e cig', 'e-cigarette', 'ecigarette', ' e cigarette', 'e-juice', ' ejuice', ' e juice', 'e-liquid', ' eliquid', ' e liquid', 'e-smoke', 'esmoke', ' e smoke', 'vape', 'vaper', 'vaping', 'vape-juice',
                    'vape-liquid', ' vapor', 'vaporizer', 'boxmod', 'cloud chaser', 'cloudchaser', 'smoke assist', 'ehookah', 'e-hookah', ' e hookah', 'stillblowingsmoke', 'still blowing smoke', 'cherry tip cigarillo', '#e-cig', '#ecig', '#e cigar', '#ejuice', '#e juice', '#eliquid', '#e liquid', '#vapor', '#ehookah', '#e hookah']
    brand_ecig = ['juul', 'vaporfi', 'vype pebble', 'v2 cig', 'v2cigs', 'v2 cigs', 'halocigs', ' njoy', 'markten', 'vuse', 'tryst', 'atomizer', 'cartomizer', 'south beach smoke', 'eversmoke', 'joye510', 'joye 510', 'joyetech', 'logicecig', 'smartsmoker', ' mistic', 'smokestiks', '21st century smoke', 'logic black label',
                  ' fin ', ' finiti', 'nicotek', 'cigirex', 'ciga&blu', 'cig&blu', 'logic&cig', 'e-swisher', 'e swisher', 'eswisher', 'ezsmoker', 'ez&cig', 'green smoke', 'cigalectric', 'xhale o2', 'xhaleo2', 'cig2o', 'green smart living', 'greensmartliving', 'krave', 'swisher blk', 'grimmgreen', '#njoy', '#fin ', '#finiti']
    policy_ecig = ['sb140', 'sb 140', 'sb24', 'sb 24']
    cessation_ecig = ['notblowingsmoke', 'not blowing smoke',
                      'tobaccofreekids', 'notareplacement']

    # test for synonym
    for keyword in synonym_ecig:
        #         print(keyword)
        if '&' not in keyword and keyword in lowercaseLine:
            return True
        elif '&' in keyword:
            k1, k2 = keyword.split('&')
            if k1 in lowercaseLine and k2 in lowercaseLine:
                return True

            # synonym_ecig_list.append(line)
            # flagHit = True
            # break

    # test for brand
    for keyword in brand_ecig:
        if keyword in lowercaseLine:
            return True
            # brand_ecig_list.append(line)
            # flagHit = True
            # break

    # test for policy
    for keyword in policy_ecig:
        if keyword in lowercaseLine:
            return True
            # policy_ecig_list.append(line)
            # flagHit = True
            # break

    # test for cessation
    for keyword in cessation_ecig:
        if keyword in lowercaseLine:
            return True
            # cessation_ecig_list.append(line)
            # flagHit = True
            # break

    return False

def getPlaceName(jsonLine):
    #     print(jsonLine['place']['name'])
    return jsonLine['place']['name']

def checkLocationInLine(line):
    try:
        tweetInJson = json.loads(line)
        tweetPlace = getPlaceName(tweetInJson)
        tweetPlace_lowercase = tweetPlace.lower()
        if ('san francisco' in tweetPlace_lowercase) or (' SF ' in tweetPlace) or ('sfo' in tweetPlace_lowercase):
            return True
    except:
        return False

def getNotECigaretteLinesFromFile(fileName):
    not_ECigarette_notInSF_Line = set()
    with open(fileName, "r") as f:
        everyLines = f.readlines()
        countOfEachFile = 0
        for line in everyLines:
            # Not in San Francisco & Not include key word
            if not checkLocationInLine(line):
                if not checkECigaretteInLine(line):
                    not_ECigarette_notInSF_Line.add(line)
                    countOfEachFile += 1
            # Each file we only take 400 lines. 24 files in total.
            if countOfEachFile >= 400:
                break
                
    # Write Not E-Cigarette in not SF area data to local file
    not_ECigarette_fileName = '/mnt/volume/result/juliana_NOT_ECigarette_2018_01.json'
    with open(not_ECigarette_fileName, "a", encoding="utf-8") as f:
        for line in not_ECigarette_notInSF_Line:
            f.write(str(line))
            
    

In [131]:
negative_directoryPath = '/mnt/volume/juliana/data/2018-01-10/'

# go through each file
files = os.listdir(negative_directoryPath)
for file in files:
#     print(file)
    file = negative_directoryPath + '/' + file
    if not os.path.isdir(file) and (os.path.splitext(file)[-1] == ".json"):
#         print(file)
        getNotECigaretteLinesFromFile(file)
        print('finish file: ', file)




finish file:  /mnt/volume/juliana/data/2018-01-10//twitter_2018-01-10_16h.json
finish file:  /mnt/volume/juliana/data/2018-01-10//twitter_2018-01-10_00h.json
finish file:  /mnt/volume/juliana/data/2018-01-10//twitter_2018-01-10_06h.json
finish file:  /mnt/volume/juliana/data/2018-01-10//twitter_2018-01-10_05h.json
finish file:  /mnt/volume/juliana/data/2018-01-10//twitter_2018-01-10_07h.json
finish file:  /mnt/volume/juliana/data/2018-01-10//twitter_2018-01-10_19h.json
finish file:  /mnt/volume/juliana/data/2018-01-10//twitter_2018-01-10_13h.json
finish file:  /mnt/volume/juliana/data/2018-01-10//twitter_2018-01-10_20h.json
finish file:  /mnt/volume/juliana/data/2018-01-10//twitter_2018-01-10_15h.json
finish file:  /mnt/volume/juliana/data/2018-01-10//twitter_2018-01-10_02h.json
finish file:  /mnt/volume/juliana/data/2018-01-10//twitter_2018-01-10_21h.json
finish file:  /mnt/volume/juliana/data/2018-01-10//twitter_2018-01-10_08h.json
finish file:  /mnt/volume/juliana/data/2018-01-10//t

Now we read the data to see how many lines in there.

In [132]:
not_ECigarette_filePath = '/mnt/volume/result/juliana_NOT_ECigarette_2018_01.json'
df_not_ECigarette_01 = pd.read_json(not_ECigarette_filePath, lines=True, encoding='utf-8') 

In [133]:
df_not_ECigarette_01[:3]

,filter_level,quote_count,retweeted,in_reply_to_screen_name,possibly_sensitive,truncated,lang,in_reply_to_status_id_str,id,extended_entities,...,source,favorited,in_reply_to_user_id,retweet_count,id_str,user,extended_tweet,quoted_status_id_str,quoted_status_id,quoted_status
0,low,0,False,CuteEmergency,0.0,False,und,9.509164e+17,951196965999607810,"{'media': [{'sizes': {'thumb': {'w': 80, 'resi...",...,"<a href=""http://twitter.com/download/android"" ...",False,568825492.0,0,951196965999607808,"{'location': 'Nashville, TN', 'default_profile...",NaN,NaN,NaN,NaN
1,low,0,False,None,0.0,True,en,NaN,951196952540008448,NaN,...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,0,951196952540008448,"{'location': 'West Coast', 'default_profile': ...",{'extended_entities': {'media': [{'sizes': {'s...,NaN,NaN,NaN
2,low,0,False,None,0.0,False,en,NaN,951196971028631553,NaN,...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,0,951196971028631552,"{'location': 'Minneapolis, MN', 'default_profi...",NaN,9.511767e+17,9.511767e+17,"{'filter_level': 'low', 'quote_count': 45, 're..."


In [134]:
df_not_ECigarette_01.columns

Index(['filter_level', 'quote_count', 'retweeted', 'in_reply_to_screen_name',
       'possibly_sensitive', 'truncated', 'lang', 'in_reply_to_status_id_str',
       'id', 'extended_entities', 'in_reply_to_user_id_str', 'timestamp_ms',
       'in_reply_to_status_id', 'created_at', 'favorite_count',
       'display_text_range', 'place', 'coordinates', 'text', 'contributors',
       'geo', 'reply_count', 'entities', 'is_quote_status', 'source',
       'favorited', 'in_reply_to_user_id', 'retweet_count', 'id_str', 'user',
       'extended_tweet', 'quoted_status_id_str', 'quoted_status_id',
       'quoted_status'],
      dtype='object')

In [135]:
df_not_ECigarette_01.text[:3]

0         @CuteEmergency Awwww https://t.co/yY8f7Y3zHc
1    This barefoot flip-flop or notices me filming ...
2    Who's going to be Ramirez's understudy now? 💅 ...
Name: text, dtype: object

In [136]:
# read the data
# filePath = "C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol/data/labeled_figure_eight_csv_1000.csv"
filePath = "/mnt/volume/labeled_data/labeled_figure_eight_csv_1000.csv"
df_labeled = pd.read_csv(filePath)
# df_labeled.is_this_posting_relevant_to_ecigarette[:3]

# Deal with ecig_labels.
ecig_labels = []
# extracting pre-labeled classification
for i in range(0,len(df_labeled.is_this_posting_relevant_to_ecigarette)):
#     print(df.is_this_posting_relevant_to_ecigarette[i])
    if df_labeled.is_this_posting_relevant_to_ecigarette[i] == 'ECIG':
        ecig_labels.append(1)
    elif df_labeled.is_this_posting_relevant_to_ecigarette[i] == 'NOT-ECIG':
        ecig_labels.append(0)
    else:
        print('Should not go to here')
# ecig_labels

# Deal with negative tweets.
print('We generated %d negative tweets.' % (len(df_not_ECigarette_01)))
label_not_ECigarette = [0]*len(df_not_ECigarette_01)
ecig_labels.extend(label_not_ECigarette)
print('We got %d labels in total, among which %d of them are positive (e-cig relevant)' % (len(ecig_labels), sum(ecig_labels)))

# Combine the two data frame
series_textFromLabeled = df_labeled.realdonaldtrump_foxandfriends_smoke_blowing_up_your_orange_ass
series_textFromGenerated = df_not_ECigarette_01.text
series_combined = series_textFromLabeled.append(series_textFromGenerated, ignore_index=True)
# df_combined = pd.concat(df_labeled.realdonaldtrump_foxandfriends_smoke_blowing_up_your_orange_ass, df_not_ECigarette_01.text)
# len(series_combined)

# from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(series_combined, ecig_labels, test_size=0.2, random_state=26)
print('length of X_train: ', len(X_train))
print('length of X_test: ', len(X_test))
print('length of y_train: ', len(y_train))
print('length of y_test: ', len(y_test))

We generated 9555 negative tweets.
We got 10557 labels in total, among which 571 of them are positive (e-cig relevant)
length of X_train:  8445
length of X_test:  2112
length of y_train:  8445
length of y_test:  2112


In [137]:
# LR
clf_ecig_LR = ECigPipeline().pipeline(LogisticRegression())
clf_ecig_LR.steps

[('tfidf',
  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                  dtype=<class 'numpy.float64'>, encoding='utf-8',
                  input='content', lowercase=True, max_df=1.0, max_features=None,
                  min_df=1, ngram_range=(1, 3), norm='l2', preprocessor=None,
                  smooth_idf=True, stop_words=None, strip_accents=None,
                  sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                  tokenizer=None, use_idf=True, vocabulary=None)),
 ('clf',
  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                     intercept_scaling=1, l1_ratio=None, max_iter=100,
                     multi_class='warn', n_jobs=None, penalty='l2',
                     random_state=None, solver='warn', tol=0.0001, verbose=0,
                     warm_start=False))]

In [138]:
clf_ecig_LR.fit(X_train, y_train)

/home/centos/anaconda3/envs/yijun/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 3), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling

In [139]:
# predict in testing set.
y_ecig_LR = clf_ecig_LR.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_ecig_LR, y_test)
f1_score_LR = computeF1Score(y_test, y_ecig_LR)
print('F1 Score: ', f1_score_LR)
auc_score_LR = computeAUC(clf_ecig_LR, X_test, y_test, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('alcohol LR AUC: ', auc_score_LR)

length of total comps:  2112
96 different item in whole dataset
accuracy: 95.454545%
F1 Score:  0.36
alcohol LR AUC:  0.9443688252870462


In [140]:
SF_filePath = '/mnt/volume/result/juliana_allSF_2018_01.json'
df_SF_01 = pd.read_json(SF_filePath, lines=True, encoding='utf-8') 

In [141]:
print('length of San Francisco data: ', len(df_SF_01))
y_SF_LR = clf_ecig_LR.predict(df_SF_01.text)
print('How many of them the classifier said is positive (e-cig relevant)?: ',sum(y_SF_LR))

length of San Francisco data:  278884
How many of them the classifier said is positive (e-cig relevant)?:  11


In [142]:
index_in_df_SF = np.where(y_SF_LR)[0]
for text in df_SF_01.iloc[index_in_df_SF,].text:
    print(text)
    print('-----------')

@Echan419 Vape lung bubbles
-----------
You want some of this Juul ?

-Me never smoking a day in my life and passing around tequila shots instead of a Juul.
-----------
You want some of this Juul ?

-Me never smoking tobacco a day in my life and passing around tequila shots instead of a Juul.
-----------
Your mcm shames me for smoking two cigarettes a day but he goes through 3 Juul pods a day
-----------
@thegreaatdane Same but with my juul.
-----------
@SweetxSauce @jnudey get a juul, you’ll look cooler
-----------
i lost my juul charger !!! 💆🏽‍♂️
-----------
BUD-DEX CBD VAPE PEN
#buddyvape #vapepens #vapelife #vapershouts #vapereviews #vapepen #vapeshop #vaping #smokeshop… https://t.co/1kcjTT2u66
-----------
My Vape fell out of my pocket on BART. Long shot, but did anyone find it? #Vape #BART #SF #SanFrancisco
-----------
Omg mango Juul pods are a game changer 😋
-----------
RIP my Green Smart Living (JUUL) hoe only lasted 2 weeks :+(
-----------


In [143]:
clf_ecig_LR.steps
LR_ecig = clf_ecig_LR.steps[1][1]
coefficients = LR_ecig.coef_.tolist()[0]
print('length of coefficients: ', len(coefficients))

length of coefficients:  152619


In [144]:
tfidf_ecig = clf_ecig_LR.get_params()['tfidf']
features = tfidf_ecig.get_feature_names()
print('length of features: ', len(features))
type(features)

length of features:  152619


list

In [145]:
featuresAndCoef = {'features':features,'coefficients':coefficients}
df_featuresAndCoef = pd.DataFrame(featuresAndCoef)
df_featuresAndCoef.sort_values(by=['coefficients'], ascending=False)

,features,coefficients
76271,juul,10.134187
138820,vape,10.018277
139248,vaping,4.995269
90920,my juul,2.253609
139340,vapor,2.137005
117120,smoke,2.116950
11873,and,2.036225
104191,pods,1.978481
90287,my,1.789591
127002,the juul,1.759060
